In [ ]:
!pip install unsloth

In [1]:
# needed to fix a bug with unsloth
%env UNSLOTH_COMPILE_DISABLE = 1

env: UNSLOTH_COMPILE_DISABLE=1


In [2]:
from unsloth import FastLanguageModel, FastModel
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import os
import torch
from torch import tensor
import torch.nn.functional as F
from transformers import TrainingArguments, Trainer, ModernBertModel, AutoModelForSequenceClassification, training_args
from datasets import load_dataset, Dataset
from tqdm import tqdm

model_name = 'answerdotai/ModernBERT-large'

NUM_CLASSES = 3
DATA_DIR = "data/"

model, tokenizer = FastModel.from_pretrained(
    model_name = model_name,load_in_4bit = False,
    max_seq_length = 2048,
    dtype = None,
    auto_model = AutoModelForSequenceClassification,
    num_labels = NUM_CLASSES,
)
print("model parameters:" + str(sum(p.numel() for p in model.parameters())))

# make all parameters trainable
for param in model.parameters():
    param.requires_grad = True

ModuleNotFoundError: No module named 'unsloth'

The dataset can be found [here](https://github.com/timothelaborie/text_classification_scripts/blob/main/data/finance_sentiment_multiclass.csv)

In [ ]:
data = pd.read_csv(DATA_DIR + "finance_sentiment_multiclass.csv")

labels = data["label"].tolist()
labels = [x-1 for x in labels]
# convert labels to one hot vectors
labels = np.eye(NUM_CLASSES)[labels]

train_data,val_data, train_labels, val_labels = train_test_split(data["text"], labels, test_size=0.1, random_state=42)
dataset = Dataset.from_list([{'text': text, 'labels': label} for text, label in zip(train_data, train_labels)])
val_dataset = Dataset.from_list([{'text': text, 'labels': label} for text, label in zip(val_data, val_labels)])

def tokenize_function(examples):
    return tokenizer(examples['text'])

dataset = dataset.map(tokenize_function, batched=True)
val_dataset = val_dataset.map(tokenize_function, batched=True)
dataset

In [ ]:
trainer = Trainer(
    model=model,
    tokenizer=tokenizer,
    train_dataset=dataset,
    eval_dataset=val_dataset,
    args=TrainingArguments(
        per_device_train_batch_size=32,
        gradient_accumulation_steps=1,
        warmup_steps=10,
        fp16=not torch.cuda.is_bf16_supported(),
        bf16=torch.cuda.is_bf16_supported(),
        optim=training_args.OptimizerNames.ADAMW_TORCH,
        learning_rate=5e-5,
        weight_decay=0.001,
        lr_scheduler_type="cosine",
        seed=3407,
        num_train_epochs=3, # bert-style models usually need more than 1 epoch
        save_strategy="epoch",

        # report_to="wandb",
        report_to="none",

        group_by_length=True,

        # eval_strategy="no",
        eval_strategy="steps",
        eval_steps=0.25,
        logging_strategy="steps",
        logging_steps=0.25,

    ),
    compute_metrics=lambda eval_pred: { "accuracy": accuracy_score(eval_pred[1].argmax(axis=-1), eval_pred[0].argmax(axis=-1)) }
)
trainer_stats = trainer.train()

In [ ]:
model = model.cuda()
model = model.eval()
FastLanguageModel.for_inference(model)
print()

In [ ]:
batch_size = 32
correct = 0
results = []

# If the val_labels are one-hot, convert to class indices
if isinstance(val_labels, np.ndarray) and val_labels.ndim == 2:
    val_true_labels = np.argmax(val_labels, axis=1)
else:
    val_true_labels = val_labels

val_texts = list(val_data)
val_true_labels = list(val_true_labels)

with torch.no_grad():
    for i in tqdm(range(0, len(val_texts), batch_size), desc="Evaluating"):
        batch_texts = val_texts[i:i+batch_size]
        batch_labels = val_true_labels[i:i+batch_size]
        # Tokenize
        inputs = tokenizer(batch_texts, return_tensors="pt", padding=True, truncation=True, max_length=2048)
        inputs = {k: v.cuda() for k, v in inputs.items()}
        # Forward pass
        outputs = model(**inputs)
        logits = outputs.logits
        probs = F.softmax(logits, dim=-1)
        preds = torch.argmax(probs, dim=-1).cpu().numpy()
        # Count correct
        correct += np.sum(preds == batch_labels)
        # Store results for display
        for j in range(len(batch_texts)):
            results.append({
                "text": batch_texts[j][:200],
                "true": batch_labels[j],
                "pred": preds[j],
                "probs": probs[j].detach().float().cpu().numpy(),
                "ok": preds[j] == batch_labels[j]
        })

accuracy = 100 * correct / len(val_texts)
print(f"\nValidation accuracy: {accuracy:.2f}% ({correct}/{len(val_texts)})")

# Show a few random samples
import random
display = 20
print("\n--- Random samples ---")
for s in random.sample(results, min(display, len(results))):
    print(f"\nText: {s['text']}")
    print(f"True: {s['true']}  Pred: {s['pred']} {'✅' if s['ok'] else '❌'}")
    print("Probs:", ", ".join([f"{k}: {v:.3f}" for k, v in enumerate(s['probs'])]))

In [ ]:
# stop running all cells
1/0

# to load the model again (run every cell above the one where the trainer is called)

In [ ]:
from transformers.trainer_utils import get_last_checkpoint

output_dir = "trainer_output"
last_checkpoint = get_last_checkpoint(output_dir)
print("Last checkpoint:", last_checkpoint)

model, tokenizer = FastModel.from_pretrained(
    model_name = last_checkpoint,load_in_4bit = False,
    max_seq_length = 2048,
    dtype = None,
    auto_model = AutoModelForSequenceClassification,
    num_labels = NUM_CLASSES,
)

In [ ]:
from torch import tensor
print(model(input_ids=tensor([[1,2,3,4,5]]).cuda(), attention_mask=tensor([[1,1,1,1,1]]).cuda()))